In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import copy as cp
import pandas as pd
import countryinfo
import geopandas as gpd
from timeit import default_timer as timer # try to measure time
from climada.engine import Impact



In [2]:
##function definition
from functions import *
from climada_functions import *
from constants import * 

In [4]:
## select data
#select variable (cmip6 naming)
selvar = 'sfcWindmax'
pathinvar = pathcmip6+'sfcWindmax/'

#preprocessing 
gst_fact = 1.67
qt = 0.98
cut=5E5
min_lat=30
max_lat=75
min_lon=-30
max_lon=30

##climada constants
haz_type = 'WS'
haz_id = 1

## naming
#name base (meteo) variable
metvar = [cmip6vars[selvar]]
spaceres = ["br_rg"] #base resolution regridded
timeres = ["day"]
domain = ["EU"]
season = ["winE"]
scen = ["allscens"]
sep = "_"
lst_bn = metvar+spaceres+timeres+domain+season
basenamemet = sep.join(lst_bn)

#preproc field
processings = ["qt"+str(qt)[-2:]+"pst","cutarea-"+format(cut,'.0E').replace("+0",''),"gst1-67"]
basenamemet_proc = make_fn(processings,basenamemet)

# Regional

In [79]:
#select simulation
modlist = modlist_allscen
regions = ['UK','WEU','SEU','NOR','EEU'] #leave empty for EU
periods = ['historical','ssp585']
impf_namelist = ['CubEOT','Em2011','Wk2021']
caltypes = ['AAI_EU',"AAI_EMDAT"]

In [80]:
#initiate df to save results
metrics = ['aai_agg',30]
metnames = ["AAI_agg","30 yr impact"]
iterables = [metnames,periods,impf_namelist]
col_idx1 = pd.MultiIndex.from_product(iterables,names=["metric","period","Impf"])
res_df = pd.DataFrame(columns=col_idx1,index=modlist)  


In [81]:
# import impact data
dict_cal = {}
df_list =[]
for caltype in caltypes:
    dict_reg = {}
    df_list =[]
    for reg in regions:
        #initiate dfs
        res_df = pd.DataFrame(columns=col_idx1,index=modlist)  
        
        for modid, modname in enumerate(modlist):
            for period in periods:
                for impf in impf_namelist:
    
                    #make file names
                    impfn = make_fn(['imp',reg,period,modname,impf,caltype],basenamemet_proc,filetype='.csv')
                    
                    #import impacts
                    imp = Impact()
                    try:
                        imp = imp.from_csv(pathimp+"impact csv/regional/stacked/"+reg+"/"+period+"/"+impfn)
                    except FileNotFoundError:
                        impfn = make_fn(['imp','stacked',reg,period,modname,impf,caltype],basenamemet_proc,filetype='.csv')
                        imp = imp.from_csv(pathimp+"impact csv/regional/stacked/"+reg+"/"+period+"/"+impfn)

                    for idm, metric in enumerate(metrics):
                        if metric == 'aai_agg':
                            stat = imp.aai_agg
                        else:  
                            stat = imp.calc_freq_curve(return_per=metric).impact
                        res_df.loc[modname,(metnames[idm],period,impf)] = stat
                        #reg_df.loc[modname,(reg,metnames[idm],period,impf)] = stat
        
        res_df = res_df.astype(np.float64)
        #reg_df = reg_df.astype(np.float64)
        df_list.append(res_df)
    
        if reg == '':
            reg = 'EU'
        dict_reg[reg] = [res_df]
    reg_df = pd.concat(df_list,keys=regions,axis=1,names=("Regions","metric","scenario","Impf"))
    dict_cal[caltype] = reg_df

In [82]:
reg_df

Regions                     UK                                            \
metric                 AAI_agg                                             
scenario            historical                                    ssp585   
Impf                    CubEOT        Em2011        Wk2021        CubEOT   
CanESM5           9.843765e+07  9.675882e+08  9.915745e+08  4.242944e+07   
CNRM-CM6-1        3.796597e+08  5.392157e+08  4.649774e+08  7.950469e+08   
CNRM-ESM2-1       3.414687e+08  6.815516e+08  6.357424e+08  4.296246e+08   
EC-Earth3-Veg     3.510634e+08  1.432814e+09  1.577194e+09  1.468508e+08   
EC-Earth3-Veg-LR  3.668507e+08  1.267993e+09  1.341422e+09  7.622050e+07   
IPSL-CM6A-LR      2.078136e+08  1.013489e+09  1.123086e+09  3.838481e+08   
MIROC-ES2L        2.603831e+08  1.415290e+09  1.424865e+09  2.060016e+08   
UKESM1-0-LL       1.090919e+08  1.282609e+09  1.223464e+09  9.822110e+07   
MRI-ESM2-0        2.438391e+08  6.636494e+08  5.391811e+08  5.218967e+08   
FGOALS-g3         3.205405e+08  9.027313e+08  9.548652e+08  3.162606e+08   
ACCESS-ESM1-5     1.832138e+08  1.142614e+08  1.376173e+08  3.821190e+08   
MIROC6            2.121772e+08  1.433306e+09  1.272610e+09  2.049053e+08   
MPI-ESM1-2-LR     2.353301e+08  5.462409e+08  5.439457e+08  2.492832e+08   
KACE-1-0-G        2.378856e+08  7.068384e+08  7.377577e+08  2.542794e+07   

Regions                                                                   \
metric                                        30 yr impact                 
scenario                                        historical                 
Impf                    Em2011        Wk2021        CubEOT        Em2011   
CanESM5           5.262931e+08  5.007782e+08  5.657856e+08  2.593639e+09   
CNRM-CM6-1        1.110797e+09  9.450372e+08  1.628812e+09  1.758312e+09   
CNRM-ESM2-1       9.359524e+08  8.889277e+08  1.722093e+09  1.605783e+09   
EC-Earth3-Veg     9.477630e+08  9.977754e+08  1.564965e+09  3.709757e+09   
EC-Earth3-Veg-LR  3.771265e+08  4.067378e+08  2.254716e+09  4.119454e+09   
IPSL-CM6A-LR      2.269905e+09  2.369302e+09  1.040767e+09  2.748015e+09   
MIROC-ES2L        1.159086e+09  1.040955e+09  1.033837e+09  4.038988e+09   
UKESM1-0-LL       1.005957e+09  9.601769e+08  6.023668e+08  4.344155e+09   
MRI-ESM2-0        1.721458e+09  1.549335e+09  1.507009e+09  1.153744e+09   
FGOALS-g3         9.601100e+08  9.834678e+08  1.250461e+09  1.568165e+09   
ACCESS-ESM1-5     1.855541e+08  2.357254e+08  7.217051e+08  4.124327e+08   
MIROC6            8.860297e+08  7.634079e+08  6.511913e+08  6.004112e+09   
MPI-ESM1-2-LR     6.349573e+08  5.701608e+08  1.073167e+09  1.346148e+09   
KACE-1-0-G        2.436343e+08  2.431589e+08  1.168200e+09  3.162282e+09   

Regions                                       ...           EEU                \
metric                                        ...       AAI_agg                 
scenario                              ssp585  ...    historical        ssp585   
Impf                    Wk2021        CubEOT  ...        Wk2021        CubEOT   
CanESM5           3.728883e+09  2.227858e+08  ...  1.275041e+07  1.972045e+08   
CNRM-CM6-1        1.858894e+09  2.434936e+09  ...  3.567189e+06  5.364728e+08   
CNRM-ESM2-1       1.956136e+09  2.039286e+09  ...  5.687168e+06  1.365554e+08   
EC-Earth3-Veg     6.138671e+09  6.016672e+08  ...  5.562017e+06  2.121449e+08   
EC-Earth3-Veg-LR  4.826401e+09  6.088374e+08  ...  2.641808e+06  2.352295e+08   
IPSL-CM6A-LR      5.274565e+09  8.101995e+08  ...  1.341915e+07  3.171023e+08   
MIROC-ES2L        6.866058e+09  6.806100e+08  ...  2.002465e+07  1.495200e+07   
UKESM1-0-LL       4.417468e+09  7.726527e+08  ...  6.875790e+06  2.291976e+08   
MRI-ESM2-0        1.253560e+09  1.583957e+09  ...  5.430578e+05  6.443949e+08   
FGOALS-g3         1.983731e+09  1.279341e+09  ...  1.969549e+07  1.960799e+08   
ACCESS-ESM1-5     4.600128e+08  1.036720e+09  ...  2.761799e+07  2.268126e+08   
MIROC6            5.445971e+09  1.498325e+09  .

In [144]:
## compute change in AAI per region per calibration 
idx=pd.IndexSlice
AAI_EU = dict_cal["AAI_EU"].loc[:,idx[:,"AAI_agg","ssp585"]]
AAI_EMDAT = dict_cal["AAI_EMDAT"].loc[:,idx[:,"AAI_agg","ssp585"]]
AAI_EU.columns = AAI_EU.columns.droplevel((1,2))
AAI_EMDAT.columns = AAI_EMDAT.columns.droplevel((1,2))

AAI_EU_mean = AAI_EU.mean()     
AAI_EMDAT_mean = AAI_EMDAT.mean()     

AAI_diff_reg_mean = AAI_EMDAT_mean.sub(AAI_EU_mean)
AAI_diff_rel_reg_mean = 100*AAI_diff_reg_mean.div(AAI_EU_mean)
AAI_diff_rel_reg_mean = pd.DataFrame(AAI_diff_rel_reg_mean).unstack().droplevel(0,axis=1)

In [145]:
AAI_diff_rel_reg_mean

Impf,CubEOT,Em2011,Wk2021
Regions,,,
UK,5.336437,5.765803,8.037361
WEU,4.928205,4.843594,6.579001
SEU,5.058717,7.113423,9.378773
NOR,4.217731,6.682957,9.026654
EEU,4.936756,5.948673,8.440207


In [147]:
def style_10high(v, props=''):
    return props if v > 5 else None
AAI_diff_rel_reg_mean.index.name = 'Regional change in AAI (%)'
s2 = AAI_diff_rel_reg_mean.style.applymap(style_10high, props='color:red;')
s2.format('{:.2f}')

Impf,CubEOT,Em2011,Wk2021
Regional change in AAI (%),,,
UK,5.34,5.77,8.04
WEU,4.93,4.84,6.58
SEU,5.06,7.11,9.38
NOR,4.22,6.68,9.03
EEU,4.94,5.95,8.44


# Aggregated

In [13]:
#select simulation

modlist = modlist_allscen + modlist_ssp585
nmods = len(modlist)
histname = 'historical'
simname = 'stacked'
periods = ['historical']
impf_namelist = ['CubEOT']
caltype = 'AAI_EMDAT'

In [14]:
#initiate df to save results
metrics = ['aai_agg','aai_agg_flt']
metnames = metrics
iterables = [metnames,periods,impf_namelist]
col_idx1 = pd.MultiIndex.from_product(iterables,names=["metric","period","Impf"])
res_df = pd.DataFrame(columns=col_idx1,index=modlist)  


In [15]:
# import impact data
dict_res = {}
flt = 100E6
#initiate dfs
res_df = pd.DataFrame(columns=col_idx1,index=modlist)  

for modid, modname in enumerate(modlist):
    for period in periods:
        for impf in impf_namelist:
            #make file names
            impfn = make_fn(['imp',simname,period,modname,impf,caltype],basenamemet_proc,filetype='.csv')
            
            #import impacts
            imp = Impact()
            imp = imp.from_csv(pathimp+"impact csv/aggregated/stacked"+"/"+period+"/"+impfn)
            #imp.imp_mat = imp.read_sparse_csr(impmatfn)
            for idm, metric in enumerate(metrics):
                if metric == 'aai_agg':
                    stat = imp.aai_agg
                elif metric == 'aai_agg_flt':
                    #imp_flt = cp.deepcopy(imp)
                    imp_ae = imp.at_event
                    mask = np.where(imp_ae>flt)
                    imp_ae_flt = imp_ae[mask]
                    aai_agg_flt = np.sum(imp_ae_flt)*imp.frequency[0]
                    #imp_flt.at_event = imp_ae[mask]
                    stat = aai_agg_flt
                    
                else:  
                    stat = imp.calc_freq_curve(return_per=metric).impact
                res_df.loc[modname,(metnames[idm],period,impf)] = stat
                #reg_df.loc[modname,(reg,metnames[idm],period,impf)] = stat

res_df = res_df.astype(np.float64)


In [16]:
res_df

metric,aai_agg,aai_agg_flt
period,historical,historical
Impf,CubEOT,CubEOT
CanESM5,4.065125e+08,2.323530e+08
CNRM-CM6-1,4.199243e+08,1.969012e+08
CNRM-ESM2-1,3.419354e+08,1.281483e+08
EC-Earth3-Veg,3.917721e+08,1.961076e+08
EC-Earth3-Veg-LR,2.818110e+08,9.548349e+07
IPSL-CM6A-LR,2.967083e+08,1.520963e+08
MIROC-ES2L,1.305402e+08,4.464455e+07
UKESM1-0-LL,3.327468e+08,1.438404e+08


In [9]:
# compute difference dropping low impact events
idx=pd.IndexSlice
AAI_fut = res_df.loc[:,idx[:,"ssp585",:]]
AAI_past = res_df.loc[:,idx[:,"historical",:]]
AAI_diff = AAI_past - AAI_fut.values
AAI_diff_rel = 100*AAI_diff.div(AAI_past)

KeyError: 'ssp585'

In [102]:
AAI_diff_rel

metric               AAI_agg                           30yrs rp              \
period            historical                         historical               
Impf                  CubEOT     Em2011      Sw2010      CubEOT      Em2011   
CanESM5            49.390177  47.892621   39.422600   31.627660    3.967661   
CNRM-CM6-1        -68.540837 -50.981557  -56.421178  -71.039485  -71.043486   
CNRM-ESM2-1         9.081620   4.800265  -14.647912  -10.411637   -3.518146   
EC-Earth3-Veg      16.725769  32.426925   31.933507  -29.648475    5.568753   
EC-Earth3-Veg-LR   16.290299  34.641967   36.385418  -22.083531   29.468247   
IPSL-CM6A-LR      -21.712627 -31.708055  -34.562058  -61.244203  -51.568281   
MIROC-ES2L         75.073747  66.956442   75.733608   71.308796   59.849811   
UKESM1-0-LL        46.139517  50.801148   36.743915  -43.903024   -2.324587   
MRI-ESM2-0       -101.182888 -57.893530  -29.308886  -98.191090  -44.388823   
FGOALS-g3          19.083009  17.974031   14.576490  -38.258538  -35.469122   
ACCESS-ESM1-5       2.145673  -3.281557   -8.114919  -87.359130    4.243839   
MIROC6             23.097036  32.686910   34.811579  -45.166869  -38.412881   
MPI-ESM1-2-LR      20.093422  13.944299   10.215416   -0.273790  -21.395381   
KACE-1-0-G         67.675466  70.636725   73.793779   62.321260   73.653072   
AWI-CM-1-1-MR      -9.165785 -14.328690  -48.311932 -109.821352  -23.620374   
BCC-CSM2-MR       -57.696418 -39.807317  -30.688017   -5.705686    3.198228   
CNRM-CM6-1-HR     -44.415669 -43.332528 -134.932700  -97.039052  -93.967441   
EC-Earth3-CC      -22.648699   8.132777   -2.395381 -148.921792  -33.387872   
HadGEM3-GC31-LL    -4.767319  -2.518422  -31.398758 -135.418984  -94.181602   
GISS-E2-1-G       -56.742001 -41.741174  -75.119762  -50.443862  -32.924320   
GFDL-CM4          -96.959169 -77.150693 -209.243062 -434.463017 -573.755756   
CMCC-CM2-SR5     -113.375237 -95.121698 -152.720500 -119.875163 -326.628635   
CMCC-ESM2         -37.467362 -33.871048  -51.832526 -168.896000 -130.428920   
HadGEM3-GC31-MM    -7.483867 -12.423011  -28.726211    5.345275  -18.504727   
NESM3             -58.208396 -53.260812  -61.645845  -29.130475   -9.177397   
MPI-ESM1-2-HR     -13.601616 -13.771975  -59.422092  -75.530779  -87.970861   
INM-CM4-8          22.677362  11.710642   12.673903    2.733473   24.880216   
INM-CM5-0         -17.190947 -39.568937  -30.472885   52.755859   24.227626   
ACCESS-CM2         -6.795109 -34.750544 -110.507700  -37.172914 -271.152503   

metric                         
period                         
Impf                   Sw2010  
CanESM5             -2.399342  
CNRM-CM6-1         -84.922028  
CNRM-ESM2-1        -21.794688  
EC-Earth3-Veg       25.498087  
EC-Earth3-Veg-LR    46.563961  
IPSL-CM6A-LR       -25.275947  
MIROC-ES2L          82.253754  
UKESM1-0-LL        -19.106430  
MRI-ESM2-0          -1.023764  
FGOALS-g3          -61.359596  
ACCESS-ESM1-5       -6.202750  
MIROC6              26.748449  
MPI-ESM1-2-LR      -28.153587  
KACE-1-0-G          81.278290  
AWI-CM-1-1-MR      -22.643106  
BCC-CSM2-MR         40.519587  
CNRM-CM6-1-HR     -277.873830  
EC-Earth3-CC       -31.178994  
HadGEM3-GC31-LL   -119.323819  
GISS-E2-1-G        -89.228281  
GFDL-CM4         -1678.235638  
CMCC-CM2-SR5      -465.728620  
CMCC-ESM2         -302.265094  
HadGEM3-GC31-MM    -81.948067  
NESM3              -11.190900  
MPI-ESM1-2-HR     -176.725065  
INM-CM4-8           38.797337  
INM-CM5-0          -30.313557  
ACCESS-CM2        -136.498495

In [91]:
#AAI_diff_rel.columns = AAI_diff_rel.columns.droplevel(1)
def style_10high(v, props=''):
    return props if v > 5 else None
AAI_diff_rel.index.name = 'Change in metric (%)'
s2 = AAI_diff_rel.style.applymap(style_10high, props='color:red;')
s2.format('{:.2f}')

In [ ]:
#initiate df to save results
metrics = ['aai_agg',1,10]
metnames = ["AAI_agg", str(1)+'yr RPI',str(10)+'yr RPI']
cutidx = ["cutarea"+format(cut,'.0E').replace("+0",'') for cut in cuts]
res_df = pd.DataFrame(columns=metnames,index=cutidx)

In [9]:
## compare processings

#select simulation
modlist = modlist_allscen+modlist_ssp585
nmods = len(modlist)
histname = 'historical'
simname = 'stacked'
periods = ['historical']
impf_namelist = ['CubEOT','Em2011','Sw2010']
procs = [1E5,5E5,1E6]
caltype = 'AAI_EMDAT_100mn'

#initiate df to save results
metrics = ["aai_agg",1,10]
metnames = ["aai_agg",str(1)+'yr_RPI',str(10)+'yr_RPI']
iterables = [metnames,periods,impf_namelist]
col_idx1 = pd.MultiIndex.from_product(iterables,names=["metric","period","Impf"])
dict_res = {}

#initiate df to save results v2
metrics = ['aai_agg',1,10]
metnames = ["AAI_agg", str(1)+'yr_RPI',str(10)+'yr_RPI']
procidx = ["cutarea"+format(proc,'.0E').replace("+0",'') for proc in procs]
iterables = [procidx,metnames,periods,impf_namelist]
col_idx2 = pd.MultiIndex.from_product(iterables,names=["proc","metric","period","Impf"])
res_df2 = pd.DataFrame(columns=col_idx2,index=modlist)

for proc in procs:
    res_df = pd.DataFrame(columns=col_idx1,index=modlist)  
    for period in periods:
        for modname in modlist:
            for impf in impf_namelist:
                if impf == 'CubEOT':
                    proc_type = 'scale_qt'
                else:
                    proc_type = 'diff_qt'
                #make file names
                processings = ["qt"+str(qt)[-2:]+"pst","cutarea"+format(proc,'.0E').replace("+0",''),"gst1-67"]
                basenamemet_proc = make_fn(processings,basenamemet)
                impfn = make_fn(['imp',simname,period,modname,impf,caltype],basenamemet_proc,filetype='.csv')
                
                #import impacts
                imp = Impact()
                try:
                    imp = imp.from_csv(pathimp+"impact csv/aggregated/stacked"+"/"+period+"/"+impfn)
                except FileNotFoundError:
                    impfn = make_fn(['imp',simname,period,modname,impf,caltype,proc_type],basenamemet_proc,filetype='.csv')
                    imp = imp.from_csv(pathimp+"impact csv/aggregated/stacked"+"/"+period+"/"+impfn)
                    
                #imp.imp_mat = imp.read_sparse_csr(impmatfn)
                for idm, metric in enumerate(metrics):
                    if metric == 'aai_agg':
                        stat = imp.aai_agg
                    else:  
                        stat = imp.calc_freq_curve(return_per=metric).impact
                    res_df.loc[modname,(metnames[idm],period,impf)] = stat
                    res_df2.loc[modname,("cutarea"+format(proc,'.0E').replace("+0",''),metnames[idm],period,impf)] = stat
    res_df = res_df.astype(np.float64)
    dict_res[proc] = res_df



In [51]:
proc0 = procidx[1]
met_diff = res_df2.copy()
met_diff.loc[:,[procidx[0]]] = met_diff[[procidx[0]]] - met_diff[proc0].values
met_diff.loc[:,[procidx[2]]] = met_diff[[procidx[2]]] - met_diff[proc0].values
met_diff.loc[:,[procidx[0]]] = 100*met_diff[[procidx[0]]] / met_diff[proc0].values
met_diff.loc[:,[procidx[2]]]= 100*met_diff[[procidx[2]]] / met_diff[proc0].values

met_diff.columns = met_diff.columns.droplevel(2)

In [62]:
pd.DataFrame(met_diff.drop(proc0,axis=1,level=0).mean()).unstack(["metric"])

0                      
metric              10yr_RPI    1yr_RPI    AAI_agg
proc       Impf                                   
cutarea1E5 CubEOT   1.239035   2.559439   6.771882
           Em2011  -2.998566  -2.862626   3.956183
           Sw2010  -0.405011  -0.180215   2.648982
cutarea1E6 CubEOT -30.800252 -33.845361 -40.303157
           Em2011  37.738760  34.427729  15.372825
           Sw2010 -13.681809 -17.260620 -23.019824

In [61]:
pd.DataFrame(met_diff.drop(proc0,axis=1,level=0).mean()))

0
proc       metric   Impf             
cutarea1E5 AAI_agg  CubEOT   6.771882
                    Em2011   3.956183
                    Sw2010   2.648982
           1yr_RPI  CubEOT   2.559439
                    Em2011   2.862626
                    Sw2010   0.180215
           10yr_RPI CubEOT   1.239035
                    Em2011   2.998566
                    Sw2010   0.405011
cutarea1E6 AAI_agg  CubEOT  40.303157
                    Em2011  15.372825
                    Sw2010  23.019824
           1yr_RPI  CubEOT  33.845361
                    Em2011  34.427729
                    Sw2010  17.260620
           10yr_RPI CubEOT  30.800252
                    Em2011  37.738760
                    Sw2010  13.681809

In [85]:
# compute difference in calibrations
idx=pd.IndexSlice
AAI1= dict_res[1E6].loc[:,idx[:,"historical"]]
AAI2 = dict_res[1E6].loc[:,idx[:,"historical"]]
AAI_diff = AAI1.sub(AAI2)
AAI_diff_rel = 100*AAI_diff.div(AAI1)
AAI_ratio = 100*AAI2.div(AAI1)

In [86]:
AAI_ratio.loc["mean"] = AAI_ratio.mean()

In [87]:
AAI_ratio.name = "Ratio captured vs missed impact events"

In [88]:
AAI_ratio.columns = AAI_ratio.columns.droplevel(1)
def style_10high(v, props=''):
    return props if v > 20 else None
AAI_ratio.index.name = 'Change in metric (%)'
s2 = AAI_ratio.style.applymap(style_10high, props='color:red;')
s2.set_caption("Ratio captured vs missed impact events")
s2.format('{:.2f}')

In [69]:
s2.title

'Ratio captured vs missed impact events'

In [38]:
AAI_diff_rel.columns = AAI_diff_rel.columns.droplevel(1)
def style_10high(v, props=''):
    return props if v > 5 else None
AAI_diff_rel.index.name = 'Change in metric(%)'
s2 = AAI_diff_rel.style.applymap(style_10high, props='color:red;')
s2.format('{:.2f}')